In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [83]:
import warnings
warnings.filterwarnings('ignore')

#### There are sonar signals that are reflected by metal cylinder and that reflected from rock. Create a model to judtge if it is refected from metal or rock.

In [84]:
df = pd.read_csv("./sonar_dataset.csv", header=None)
#if you specify header=None it takes the integer values as column names
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
182,0.0095,0.0308,0.0539,0.0411,0.0613,0.1039,0.1016,0.1394,0.2592,0.3745,...,0.0181,0.0019,0.0102,0.0133,0.0040,0.0042,0.0030,0.0031,0.0033,M
86,0.0188,0.0370,0.0953,0.0824,0.0249,0.0488,0.1424,0.1972,0.1873,0.1806,...,0.0093,0.0033,0.0113,0.0030,0.0057,0.0090,0.0057,0.0068,0.0024,R
120,0.0346,0.0509,0.0079,0.0243,0.0432,0.0735,0.0938,0.1134,0.1228,0.1508,...,0.0040,0.0122,0.0107,0.0112,0.0102,0.0052,0.0024,0.0079,0.0031,M
161,0.0305,0.0363,0.0214,0.0227,0.0456,0.0665,0.0939,0.0972,0.2535,0.3127,...,0.0200,0.0070,0.0070,0.0086,0.0089,0.0074,0.0042,0.0055,0.0021,M
110,0.0210,0.0121,0.0203,0.1036,0.1675,0.0418,0.0723,0.0828,0.0494,0.0686,...,0.0117,0.0101,0.0061,0.0031,0.0099,0.0080,0.0107,0.0161,0.0133,M


In [85]:
df.shape

(208, 61)

In [86]:
df.columns

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60],
      dtype='int64')

In [87]:
df[60].value_counts() #last column

60
M    111
R     97
Name: count, dtype: int64

In [88]:
X = df.drop(60, axis='columns')
y = df[60]
y.sample(10)

85     R
9      R
206    M
139    M
44     R
147    M
121    M
59     R
168    M
45     R
Name: 60, dtype: object

In [89]:
y.replace({'M': 0 ,'R':1}, inplace=True) 
y.sample(10)

167    0
74     1
0      1
187    0
202    0
175    0
63     1
26     1
204    0
160    0
Name: 60, dtype: int64

# Train test split

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=5)

In [91]:
X_train.shape, X_test.shape

((156, 60), (52, 60))

#Train

In [92]:
import tensorflow as tf
from tensorflow import keras

In [93]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'), 
    keras.layers.Dense(30, activation='relu'), 
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'), 
])

model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8) 
# batch size 8= mini batch gradient descent

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.6907 - accuracy: 0.5449 
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6718 - accuracy: 0.6603
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6544 - accuracy: 0.6603
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6376 - accuracy: 0.6923
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6136 - accuracy: 0.6731
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5793 - accuracy: 0.7372
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5632 - accuracy: 0.7051
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5234 - accuracy: 0.7436
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4883 - accuracy: 0.8013
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4629 - accuracy: 0.8013
Epoch 11

* The accuracy is 1, the model looks too good
* But the accuracy does not transfer well to test data

In [94]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 6ms/step - loss: 0.5450 - accuracy: 0.8846


[0.5449550747871399, 0.8846153616905212]

In [110]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid'), 
])
#dropout layers are generally added after the hidden layers
modeld.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=200, batch_size=8) 
# batch size 8= mini batch gradient descent

Epoch 1/200
20/20 [==============================] - 1s 1ms/step - loss: 0.7461 - accuracy: 0.4744
Epoch 2/200
20/20 [==============================] - 0s 2ms/step - loss: 0.7084 - accuracy: 0.4808
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6988 - accuracy: 0.5064
Epoch 4/200
20/20 [==============================] - 0s 1ms/step - loss: 0.6915 - accuracy: 0.4679
Epoch 5/200
20/20 [==============================] - 0s 1ms/step - loss: 0.7079 - accuracy: 0.5192
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6951 - accuracy: 0.5513
Epoch 7/200
20/20 [==============================] - 0s 1ms/step - loss: 0.6978 - accuracy: 0.5256
Epoch 8/200
20/20 [==============================] - 0s 1ms/step - loss: 0.6820 - accuracy: 0.5321
Epoch 9/200
20/20 [==============================] - 0s 1ms/step - loss: 0.6768 - accuracy: 0.5833
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6951 - accuracy: 0.4872
Epoch 11/

In [111]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.5769 - accuracy: 0.8269


[0.5768502354621887, 0.8269230723381042]